In [3]:
import cv2
import torch
import numpy as np

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

cap = cv2.VideoCapture(r"C:\Users\ASUS\Desktop\WhatsApp Video 2023-10-03 at 00.28.42_958a94ce.mp4")
cap2 = cv2.VideoCapture(r"C:\Users\ASUS\Desktop\WhatsApp Video 2023-10-03 at 00.28.41_7f0f8d8f.mp4")

# Define boundary coordinates (adjust as needed)
boundary_x1 = 1  # Top-left x-coordinate
boundary_y1 = 1  # Top-left y-coordinate
boundary_x2 = 1280  # Bottom-right x-coordinate
boundary_y2 = 720  # Bottom-right y-coordinate

# Create a dictionary to store the previous positions of each tracked person
prev_positions = {}

while True:
    # Read a new frame
    ret, frame = cap.read()
    ret2, frame2 = cap2.read()
    if not ret or not ret2:
        break

    frame_height, frame_width, _ = frame.shape
    frame_height2, frame_width2, _ = frame2.shape  # Get frame dimensions

    # Perform object detection using YOLOv5
    results = model(frame)
    results2 = model(frame2)

    persons = results.pred[0][results.pred[0][:, -1] == 0]
    persons2 = results2.pred[0][results2.pred[0][:, -1] == 0]

    # Draw directional arrows for each tracked person and count of persons
    num_persons = len(persons)
    num_persons2 = len(persons2)
    cv2.putText(frame, f'PERSONS: {num_persons}', (10, frame_height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.putText(frame2, f'PERSONS: {num_persons2}', (10, frame_height2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    for person in persons:
        x1, y1, x2, y2, conf = person[:5].cpu().numpy()
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)

        # Check if the person has a previous position recorded
        if center_x != 0 and center_y != 0:
            if center_x < boundary_x1 or center_y < boundary_y1 or center_x > boundary_x2 or center_y > boundary_y2:
                continue  # Skip persons outside the boundary

            person_id = int(person[5])
            if person_id in prev_positions:
                prev_x, prev_y = prev_positions[person_id]

                # Draw a green line arrow from the previous position to the current position
                cv2.arrowedLine(frame, (prev_x, prev_y), (center_x, center_y), (0, 255, 0), 2)

            # Store the current position as the previous position
            prev_positions[person_id] = (center_x, center_y)

            # Display confidence score
            cv2.putText(frame, f'Confidence: {conf:.2f}', (center_x, center_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    for person in persons2:
        x1, y1, x2, y2, conf = person[:5].cpu().numpy()
        center_x = int((x1 + x2) / 2)
        center_y = int((y1 + y2) / 2)

        # Check if the person has a previous position recorded
        if center_x != 0 and center_y != 0:
            if center_x < boundary_x1 or center_y < boundary_y1 or center_x > boundary_x2 or center_y > boundary_y2:
                continue  # Skip persons outside the boundary

            person_id = int(person[5])
            if person_id in prev_positions:
                prev_x, prev_y = prev_positions[person_id]

                # Draw a green line arrow from the previous position to the current position
                cv2.arrowedLine(frame2, (prev_x, prev_y), (center_x, center_y), (0, 255, 0), 2)

            # Store the current position as the previous position
            prev_positions[person_id] = (center_x, center_y)

            # Display confidence score
            cv2.putText(frame2, f'Confidence: {conf:.2f}', (center_x, center_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Draw the boundary
    cv2.rectangle(frame, (boundary_x1, boundary_y1), (boundary_x2, boundary_y2), (255, 0, 0), 2)
    cv2.rectangle(frame2, (boundary_x1, boundary_y1), (boundary_x2, boundary_y2), (255, 0, 0), 2)

    cv2.imshow('Cam 1', frame)
    cv2.imshow('Cam 2', frame2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cap2.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-30 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
